In [ ]:
from ase.io import read, write
from ase.units import Bohr, Angstrom
from GMPFeaturizer import GMPFeaturizer, ASEAtomsConverter, PymatgenStructureConverter
import numpy as np

# Need to edit paths.
# .gpsp files will be uploaded to github under folder gmp_pseudos
path_to_psp = '<your local path>/gmp_pseudos/<specify which psps>.gpsp'
traj_list = os.listdir('/storage/ice-shared/vip-vvi/nnff-data/dft/')
trajs = [read(_traj) for _traj in traj_list]
derivatives = True
sigmas = np.array([0.75, 0.92307692,  1.09615385,  1.26923077,  1.44230769,  1.61538462,  1.78846154,  1.96153846,  2.13461538,  2.30769231,  2.48076923,  2.65384615,
                   2.82692308,  3.0])*(Bohr/Angstrom)
max_order = 3
calc_gmp_desc(trajs, sigmas, max_order, path_to_psp, derivatives=False)

In [ ]:
def calc_gmp_desc(images:list, sigmas, max_order, path_to_psp, square=True, derivatives=True, cores=1):
    converter = ASEAtomsConverter()
    # converter = PymatgenStructureConverter()

    # setup featurizer - note, forced internally to only consider solid harmonics.
    GMPs = {
        "GMPs": {   
            "orders": [int(x) for x in range(max_order+1)], 
            "sigmas": sigmas.tolist(),   
        },
        # path to the pseudo potential file
        "psp_path": path_to_psp, 
        # basically the accuracy of the resulting features
        "overlap_threshold": 1e-16, 
        # whether the features are squared, 
        #no need to change if you are not considering the feature derivatives
        "square": square, 
    }
    featurizer = GMPFeaturizer(GMPs=GMPs, calc_derivatives=derivatives, verbose=True)
    # calculate features
    result = featurizer.prepare_features(images, cores=cores, converter=converter)

    # access data
    #features = [jnp.array(entry["features"]) for entry in result]
    #features = jnp.stack(features)
    features = [np.array(entry["features"]) for entry in result]
    features = np.stack(features)
    if derivatives:
        feature_primes = [entry["feature_primes"] for entry in result]
        feature_primes = np.stack(feature_primes)
    else:
        feature_primes = None
    return features, feature_primes